In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
'''
Takes too long to train, because there are too "many" features (26 of them are 1 feature). Need to clean up with PCA.
Current training rate with 1000 training examples and 11 features is 45 minutes.
'''

'\nTakes too long to train, because there are too "many" features (26 of them are 1 feature). Need to clean up with PCA.\n'

In [5]:
featured_df = pd.read_csv("featured_df.csv", encoding="utf-8")
featured_df.head()

,x,y,num_vowels,num_words,num_digits,num_symbols,num_chars,is_plural,has_char_repeat,has_color,has_brand,has_descriptor,has_unit
0,LGHT NNFT YGRT,LIGHT NONFAT YOGURT,0,3,0,0,14,0,1,1,1,0,0
1,WRHDS CNDY WRMS,WARHEADS CANDY WORMS,0,3,0,0,15,1,0,0,1,1,0
2,WGMNS MN HRT,WEGMANS MINI HEART,0,3,0,0,12,0,0,0,1,0,0
3,RSN LMND CRNCH T CLSTRS,RAISIN ALMOND CRUNCH OAT CLUSTERS,0,5,0,0,23,1,0,0,1,0,0
4,WHT MT CRNBRRY PCN CHCKN SLD,WHITE MEAT CRANBERRY PECAN CHICKEN SALAD,0,6,0,0,28,0,1,0,1,0,0


In [6]:
limit = int(105722/100)
featured_df = featured_df.head(limit)
le = LabelEncoder()
y_transform = le.fit_transform(featured_df['y'].tolist())
y_transform.shape

(1057,)

In [7]:
## for now train with all x
## for now train with all y
all_cols = featured_df.columns.values.tolist()
all_cols.remove('x')
all_cols.remove('y')

train_x = featured_df[all_cols]
train_x.head()

,num_vowels,num_words,num_digits,num_symbols,num_chars,is_plural,has_char_repeat,has_color,has_brand,has_descriptor,has_unit
0,0,3,0,0,14,0,1,1,1,0,0
1,0,3,0,0,15,1,0,0,1,1,0
2,0,3,0,0,12,0,0,0,1,0,0
3,0,5,0,0,23,1,0,0,1,0,0
4,0,6,0,0,28,0,1,0,1,0,0


In [8]:
train_x.shape

(1057, 11)

In [9]:
featured_df['label'] = y_transform
train_y = featured_df['label']
featured_df.head()

,x,y,num_vowels,num_words,num_digits,num_symbols,num_chars,is_plural,has_char_repeat,has_color,has_brand,has_descriptor,has_unit,label
0,LGHT NNFT YGRT,LIGHT NONFAT YOGURT,0,3,0,0,14,0,1,1,1,0,0,513
1,WRHDS CNDY WRMS,WARHEADS CANDY WORMS,0,3,0,0,15,1,0,0,1,1,0,996
2,WGMNS MN HRT,WEGMANS MINI HEART,0,3,0,0,12,0,0,0,1,0,0,1000
3,RSN LMND CRNCH T CLSTRS,RAISIN ALMOND CRUNCH OAT CLUSTERS,0,5,0,0,23,1,0,0,1,0,0,754
4,WHT MT CRNBRRY PCN CHCKN SLD,WHITE MEAT CRANBERRY PECAN CHICKEN SALAD,0,6,0,0,28,0,1,0,1,0,0,1018


In [10]:
train_x = np.array(train_x)
train_y = np.array(train_y)

In [11]:
gbdt = GradientBoostingClassifier(n_estimators=50, max_depth=3)

In [12]:
%%time
'''
train on everything (currently 33% at once)
'''
gbdt.fit(train_x, train_y)

Wall time: 45min 14s


In [13]:
shuffled = featured_df.sample(frac=1)
eval_df = shuffled.iloc[:1000]
preds = gbdt.predict(eval_df.drop(columns=['x', 'y', 'label'], axis=1))
preds

array([ 660,  118,  396,  896,  271,  262,  271,  896,  993,  973,  389,
        231,  786,  424,  453,  396,  337,  262,  660,  786,  348,  424,
        299,  289,  324,  289,  660,  212,  557, 1056,  396,  643,  778,
        650,  786,  512,  118,  213,  999,  228,  632,  351, 1036,  936,
        660,  914,  914,  632,  896,  424,  660,  389,  348,  896,  914,
        660,  512,  896,  627,  396,  486,  160,  389,  176,  289,  388,
        289,  887,  447,  413,  289,  183, 1036,  896,  176,  533,  583,
        896,  417,  660,  417,  529,  289,  396,  289,  660,  289,  289,
        896,  305,  417,  289,  289,  540,  348,  993,  412,  660,  417,
        284,  887,  993,  413,  305,  289,  896,  896,  632,  887, 1036,
        299,  388,  619,  887,  447,  417,  900,  632,  396,  896,  417,
        212,  289,  984,  395, 1010,  447,  722,  348,  417,  619,  335,
        305,  333,  786,  708,  660,  585,  660, 1043,  486,  171, 1036,
        887,   47,  557,  486,   47,  660,  453,  8

In [14]:
eval_df['preds'] = le.inverse_transform(preds)

c:\users\kevin\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
eval_df[['x', 'y', 'preds']]

,x,y,preds
1038,NNS HMGRWN RG CK MX CNFTT,ANNIES HMGRWN ORG CAKE MIX CONFETTI,ORGANIC OIL BASED HERB EXTRACT
545,BLCK BN VGG BRGR,BLACK BEAN VEGGIE BURGER,BLACK BEAN VEGGIE BURGER
589,V8 BVRG PMBLBRRY,V8 BEVERAGE POMBLUEBERRY,GRAPEFRUIT RIMMING SALT
736,HLD LD FSHND DTCH CC CKS,AHOLD OLD FASHIONED DUTCH COCOA COOKIES,STEAM READY BROCCOLI FLORESS
871,DTPC MDJL DTS,DATEPAC MEDJOOL DATES,DATEPAC MEDJOOL DATES
70,JC BLND FRM RGNC CNCNTRT,JUICE BLEND FROM ORGANIC CONCENTRATE,DAIRY FREE PROBIOTIC DRINKABLE YOGURT
210,DTS TRPCL DTS,DOTS TROPICAL DOTS,DATEPAC MEDJOOL DATES
88,SWTRTS GMMS,SWEETARTS GUMMIES,STEAM READY BROCCOLI FLORESS
342,STHWST CHCKN FR FJTS,SOUTHWEST CHICKEN FOR FAJITAS,VICTORIA AMORY SHERRY KETCHUP
732,CRN CRSPS,CORN CRISPS,UNIEKAAS MEADOWKAAS
